<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/TAPT_seq_bn_pretrain_test_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This is 5 epoch test of the pretraining of the TAPT for amazon helpfulness. I added earlystoppingcallback set to 5 (for 100 epochs) and set logging to epoch as well

train
*   2 epochs = 7 minutes
*   100 epochs = 5.8 hours
* Steps = 25 per epoch

eval loss
* after 2 epochs 3.2234
* eval runtime = 21 seconds




# Setup

In [1]:
from time import time
start_time = time()

In [2]:
# from google.colab import drive
# drive.mount("/content/drive")
# %cd '/content/drive/MyDrive/AdaptOrDie'

In [3]:
!pip install datasets
!pip install transformers[torch]
!pip install adapters
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 4.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86

In [4]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.4 MB/s eta 0:00:00


In [5]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

print_gpu_utilization()

GPU 448 MB


In [6]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    accuracy = accuracy_score(labels, preds)

   # Calculate precision, recall, and F1-score
    f1 = f1_score(labels, preds, average='macro')

    return {
        'accuracy': accuracy,
        'f1_macro': f1
    }

# Pretraining

In [7]:
from datasets import load_dataset

pretrain_dataset_name = "BigTMiami/amazon_helpfulness_TAPT_pretraining_dataset"

pretrain_train_dataset, pretrain_eval_dataset = load_dataset(pretrain_dataset_name, split=["test","dev"])

print(pretrain_train_dataset)
print(pretrain_eval_dataset)

Generating train split:   0%|          | 0/44876 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/9754 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/1980 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 9754
})
Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1980
})


In [8]:
from transformers import RobertaConfig
from adapters import AutoAdapterModel

config = RobertaConfig.from_pretrained("roberta-base")
pretrain_model = AutoAdapterModel.from_pretrained(
    "roberta-base",
    config=config,
)

/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaAdapterModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.weight', 'heads.default.3.bias', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
# THIS MUST BE SET TO MATCH  pretraining_adapter_type
from adapters import SeqBnConfig

pretraining_adapter_config = SeqBnConfig()
pretrained_adapter_name = "tapt_seq_bn"
pretrained_adapter_hub_name = f"pretrain_{pretrained_adapter_name}_adpater"

In [10]:
print(pretraining_adapter_config.__class__.__name__)
print(pretrained_adapter_name)

SeqBnConfig
tapt_seq_bn


In [11]:
# Add a new adapter
pretrain_model.add_adapter(pretrained_adapter_name, config=pretraining_adapter_config)

# Add head for masked language modeling
pretrain_model.add_masked_lm_head(pretrained_adapter_name)

# Set the adapter to be used for training
pretrain_model.train_adapter(pretrained_adapter_name)

print(f"Pretrained Adapter Name: {pretrained_adapter_name}")

Pretrained Adapter Name: tapt_seq_bn


In [12]:
pretrain_model.active_adapters

Stack[tapt_seq_bn]

In [13]:
summary = pretrain_model.adapter_summary()
print(summary)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
tapt_seq_bn              bottleneck          894,528       0.718       1       1
--------------------------------------------------------------------------------
Full model                               124,645,632     100.000               0


In [14]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [15]:
from transformers import DataCollatorForLanguageModeling

pretraining_data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

In [20]:
from transformers import TrainingArguments

pretraining_args = TrainingArguments(
    output_dir="tapt_helpfulness_pretraining_model",
    learning_rate=0.0005, # Paper for DAPT training
    per_device_train_batch_size=32, # 346,m - TRYING SMALLER BATCH SIZE
    per_device_eval_batch_size=32, #346,m - TRYING SMALLER BATCH SIZE
    gradient_accumulation_steps=11, # Creates effective batch of 32*11*6 = 2112
    num_train_epochs=5, # Just one epoch for one pass on data
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2, # Saves latest 2 checkpoints
    # torch_compile=True,  # Much Faster
    push_to_hub=True,
    hub_strategy="checkpoint", # Only pushes at end with save_model()
    logging_strategy="epoch", # Is default
    metric_for_best_model="eval_loss", # Is default
    load_best_model_at_end=True,
    report_to="tensorboard",
)

In [21]:
from transformers import EarlyStoppingCallback

early_stopping_callback = EarlyStoppingCallback(early_stopping_patience=5)

In [22]:
from adapters import AdapterTrainer

pretrain_trainer = AdapterTrainer(
    model=pretrain_model,
    args=pretraining_args,
    train_dataset=pretrain_train_dataset,
    eval_dataset=pretrain_eval_dataset,
    data_collator=pretraining_data_collator,
    callbacks=[early_stopping_callback],
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(


In [24]:
pretrain_results = pretrain_trainer.train()
print(pretrain_results)

Epoch,Training Loss,Validation Loss
0,9.201500,4.376653
1,3.309000,2.518785
2,2.422600,2.148256
3,2.291300,2.072328
4,2.097400,2.030768


TrainOutput(global_step=135, training_loss=3.875737508138021, metrics={'train_runtime': 1160.1436, 'train_samples_per_second': 42.038, 'train_steps_per_second': 0.116, 'total_flos': 1.2943981566738432e+16, 'train_loss': 3.875737508138021, 'epoch': 4.87})


In [25]:
import pandas as pd

pd.DataFrame(pretrain_trainer.state.log_history)

,loss,learning_rate,epoch,step,eval_loss,eval_runtime,eval_samples_per_second,eval_steps_per_second,train_runtime,train_samples_per_second,train_steps_per_second,total_flos,train_loss
0,9.2015,0.000429,0.97,27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,0.97,27,4.376653,21.6090,91.628,2.869,NaN,NaN,NaN,NaN,NaN
2,3.3090,0.000317,1.98,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,1.98,55,2.518785,21.5634,91.822,2.875,NaN,NaN,NaN,NaN,NaN
4,2.4226,0.000206,2.99,83,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,2.99,83,2.148256,21.5617,91.830,2.875,NaN,NaN,NaN,NaN,NaN
6,2.2913,0.000099,3.97,110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,3.97,110,2.072328,21.5535,91.864,2.877,NaN,NaN,NaN,NaN,NaN
8,2.0974,0.000000,4.87,135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,4.87,135,2.030768,21.5435,91.907,2.878,NaN,NaN,NaN,NaN,NaN


In [26]:
after_pretrain_eval_results = pretrain_trainer.evaluate()
after_pretrain_eval_results_loss = after_pretrain_eval_results["eval_loss"]
print(f"After pretraining loss: {after_pretrain_eval_results_loss:.4f}")
print(after_pretrain_eval_results)

After pretraining loss: 2.0261
{'eval_loss': 2.0260536670684814, 'eval_runtime': 21.5879, 'eval_samples_per_second': 91.718, 'eval_steps_per_second': 2.872, 'epoch': 4.87}


In [27]:
pretrain_model.push_adapter_to_hub(
    pretrained_adapter_hub_name,
    pretrained_adapter_name,
    datasets_tag=pretrain_dataset_name,
    overwrite_adapter_card=True,
)

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

pytorch_adapter.bin:   0%|          | 0.00/3.60M [00:00<?, ?B/s]

pytorch_model_head.bin:   0%|          | 0.00/157M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/BigTMiami/pretrain_tapt_seq_bn_adpater/commit/5d8b9b5de88724b1097e9b9e2f58408b0345586d', commit_message='Upload model', commit_description='', oid='5d8b9b5de88724b1097e9b9e2f58408b0345586d', pr_url=None, pr_revision=None, pr_num=None)

In [28]:
end_time = time()
total_time = end_time - start_time
print(f"Total Time: {total_time/60:.0f} minutes OR {total_time:.0f} seconds")

Total Time: 24 minutes OR 1464 seconds


# Classification

In [ ]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
